In [1]:
from datetime import datetime

In [2]:
import requests

my_path = '/data/vol/depart'
curl = requests.get('http://192.168.122.206:50070/webhdfs/v1/%s?op=LISTSTATUS&namenoderpcaddress=192.168.122.206:8020' % my_path)

In [3]:
curl

<Response [200]>

In [4]:
import os

file_paths = []
for file_status in curl.json()['FileStatuses']['FileStatus']:
    file_name = file_status['pathSuffix']
    # this is the file name in the queried directory
    if file_name.endswith('.csv'):
    # if statement is only required if the directory contains unwanted files (i.e. non-csvs).
        file_paths.append(os.path.join(my_path, file_name))
        # os.path.join asserts your result consists of absolute path

file_paths

['/data/vol/depart/2020-01-20_14-27-12.csv',
 '/data/vol/depart/2020-01-20_15-27-13.csv',
 '/data/vol/depart/2020-01-20_16-27-14.csv',
 '/data/vol/depart/2020-01-20_17-27-15.csv',
 '/data/vol/depart/2020-01-20_18-27-17.csv',
 '/data/vol/depart/2020-01-20_19-27-18.csv',
 '/data/vol/depart/2020-01-20_20-27-19.csv',
 '/data/vol/depart/2020-01-20_21-27-20.csv',
 '/data/vol/depart/2020-01-20_22-27-21.csv',
 '/data/vol/depart/2020-01-20_23-27-23.csv',
 '/data/vol/depart/2020-01-21_00-27-24.csv',
 '/data/vol/depart/2020-01-21_01-27-25.csv',
 '/data/vol/depart/2020-01-21_02-27-26.csv',
 '/data/vol/depart/2020-01-21_03-27-27.csv',
 '/data/vol/depart/2020-01-21_04-27-29.csv',
 '/data/vol/depart/2020-01-21_05-27-30.csv',
 '/data/vol/depart/2020-01-21_06-27-31.csv',
 '/data/vol/depart/2020-01-21_07-27-32.csv',
 '/data/vol/depart/2020-01-21_08-27-34.csv',
 '/data/vol/depart/2020-01-21_09-27-35.csv',
 '/data/vol/depart/2020-01-21_10-27-36.csv',
 '/data/vol/depart/2020-01-21_11-27-37.csv',
 '/data/vo

In [5]:
import pandas as pd

dfs = []
web_url = "http://192.168.122.206:50075/webhdfs/v1/%s?op=OPEN&namenoderpcaddress=192.168.122.206:8020"
#                                                  
#                                    Operation is now OPEN
for file_path in file_paths:
    print(file_path)
    file_url = web_url % file_path
    print(file_url)
    # http://192.168.10.1:50070/webhdfs/v1/my_path/file1.csv?op=OPEN
    dfs.append(pd.read_csv(file_url))

KeyboardInterrupt: 

In [ ]:
len(dfs)

In [ ]:
df=dfs[0]
for i in range (1,len(dfs)):
    print(i)
    df=pd.concat([df,dfs[i]],sort=False,ignore_index=True)

In [ ]:
df[df['turnflightinternalid']=='1011748425']
df=df.drop_duplicates(subset=['aircraft_aircrafttype_iatacode','timestamps_eobt','airports_next_name'], keep="last")

In [ ]:
df.dropna(subset=['timestamps_aobt'])['timestamps_aobt']

In [ ]:
df['timestamps_eobt'] = df['timestamps_eobt'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))

In [ ]:
df['timestamps_sobt'] =df['timestamps_sobt'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))

In [ ]:
len(set(df['timestamps_eobt']))

In [ ]:
myset = set(df[df['remark_description']!='None']['remark_description'])
print(myset)

In [ ]:
df=df.replace("None",pd.np.nan)

In [ ]:
len(df['flightnumbers_operatorinternalflightnumber'])

In [ ]:
df['retard']=0

In [ ]:
df['retard_depart']=0

In [ ]:
df[df['remark_description']!=1]

In [ ]:
def change(row):
    row['day']=row['timestamps_sobt'].day
    row['month']=row["timestamps_sobt"].month
    row['year']=row["timestamps_sobt"].year
    if(row['timestamps_sobt']<row['timestamps_eobt']):
        row['retard_depart']=int((row['timestamps_eobt']-row['timestamps_sobt']).seconds/60)
    else :
        row['retard_depart']=0
    if row['retard_depart']>15:
        row['retard']="1"
    else:
        row['retard']="0"
    row['timestamps_sobt']=datetime.timestamp(row['timestamps_sobt'])
    row['timestamps_eobt']=datetime.timestamp(row['timestamps_eobt'])
    return row
            

In [ ]:
df=df.apply(lambda x:change(x),axis=1)

In [ ]:
df[df["remark_description"]=="Vol retardé"][['aircraft_aircrafttype_icaocode','flightnumbers_icaoflightnumber', 'airportresources_terminal','airports_destination_name','countrytype_code','timestamps_eobt','timestamps_sobt',"retard_depart","retard"]]

In [ ]:
df.columns

In [ ]:
traindf=df[['aircraft_aircrafttype_icaocode','flightnumbers_icaoflightnumber', 'airportresources_terminal','airports_destination_icaocode','countrytype_code',"retard_depart","retard",'timestamps_sobt','timestamps_eobt','day','month','year']]

In [ ]:
len(traindf[traindf["retard"]==1])

In [ ]:
traindf.columns

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.axis('equal')
langs = ['avec retard', 'sans retard']
students = [len(traindf[traindf["retard"]==1]),len(traindf[traindf["retard"]!=1])]
ax.pie(students, labels = langs,autopct='%1.2f%%')
plt.show()

In [ ]:
plt.show()

In [ ]:
traindf.dtypes

In [ ]:
traindf.isnull().sum()

In [ ]:
traindf=traindf[['aircraft_aircrafttype_icaocode','flightnumbers_icaoflightnumber','airports_destination_icaocode','countrytype_code',"retard_depart","retard",'timestamps_sobt','day','month','year']].dropna()

In [ ]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()

traindf[['aircraft_aircrafttype_icaocode','flightnumbers_icaoflightnumber','airports_destination_icaocode','timestamps_sobt','countrytype_code']]=traindf[['aircraft_aircrafttype_icaocode','flightnumbers_icaoflightnumber','airports_destination_icaocode','timestamps_sobt','countrytype_code']].apply(lb_make.fit_transform)


In [ ]:
traindf['retard']=traindf['retard'].astype("str")

In [ ]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(traindf.drop("retard", axis=1), traindf["retard"], test_size=0.2, random_state=42)

In [ ]:
from imblearn.over_sampling import SMOTE

# Resample the minority class. You can change the strategy to 'auto' if you are not sure.
sm = SMOTE(sampling_strategy='minority', random_state=7)

# Fit the model to generate the data.
train_x,  train_y = sm.fit_sample(train_x, train_y)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=13)
model.fit(train_x,train_y)

In [ ]:
predicted = model.predict(test_x)
model.score(test_x, test_y)

In [ ]:
from sklearn.metrics import roc_auc_score
probabilities = model.predict_proba(test_x)
probabilities[:,1]

In [ ]:
roc_auc_score(test_y, probabilities[:, 1])

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_y, predicted)

In [ ]:
predicted

In [ ]:
train_y

In [ ]:
train_predictions

In [ ]:
from sklearn.metrics import precision_score

train_predictions = model.predict(train_x)
precision_score(train_y, train_predictions, pos_label="1")

In [ ]:
train_predictions

In [ ]:
from sklearn.metrics import recall_score

recall_score(train_y, train_predictions, pos_label="1")

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, _ = roc_curve(test_y, probabilities[:, 1], pos_label="1")
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], color='grey', lw=1, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier= KNeighborsClassifier(20,p=2,metric='euclidean')

In [ ]:
classifier.fit(train_x,train_y)
predicted1=classifier.predict(test_x)

In [ ]:
classifier.score(test_x,test_y)

In [ ]:
probabilities2 = classifier.predict_proba(test_x)
probabilities2

In [ ]:
roc_auc_score(test_y, probabilities2[:, 1])

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_y, predicted1)

In [ ]:
from sklearn.metrics import precision_score

train_predictions1 = classifier.predict(train_x)
precision_score(train_y, train_predictions1, pos_label="1")

In [ ]:
from sklearn.metrics import recall_score

recall_score(train_y, train_predictions1, pos_label="1")

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, _ = roc_curve(test_y, probabilities2[:, 1], pos_label="1")
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], color='grey', lw=1, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
print(_)

In [ ]:
df[df['timestamps_eobt']=='2020-01-21 08:25:00']

In [ ]:
def plot_roc_curve(fprs, tprs):
    """Plot the Receiver Operating Characteristic from a list
    of true positive rates and false positive rates."""
    
    # Initialize useful lists + the plot axes.
    tprs_interp = []
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)
    f, ax = plt.subplots(figsize=(14,10))
    
    # Plot ROC for each K-Fold + compute AUC scores.
    for i, (fpr, tpr) in enumerate(zip(fprs, tprs)):
        tprs_interp.append(np.interp(mean_fpr, fpr, tpr))
        tprs_interp[-1][0] = 0.0
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc)
        ax.plot(fpr, tpr, lw=1, alpha=0.3,
                 label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))
        
    # Plot the luck line.
    plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
             label='Luck', alpha=.8)
    
    # Plot the mean ROC.
    mean_tpr = np.mean(tprs_interp, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    ax.plot(mean_fpr, mean_tpr, color='b',
             label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
             lw=2, alpha=.8)
    
    # Plot the standard deviation around the mean ROC.
    std_tpr = np.std(tprs_interp, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                     label=r'$\pm$ 1 std. dev.')
    
    # Fine tune and show the plot.
    ax.set_xlim([-0.05, 1.05])
    ax.set_ylim([-0.05, 1.05])
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title('Receiver operating characteristic')
    ax.legend(loc="lower right")
    plt.show()
    return (f, ax)

def compute_roc_auc(index):
    y_predict = clf.predict_proba(X.iloc[index])[:,1]
    fpr, tpr, thresholds = roc_curve(y.iloc[index], y_predict)
    auc_score = auc(fpr, tpr)
    return fpr, tpr, auc_score

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

cv = StratifiedKFold(n_splits=5, random_state=123, shuffle=True)
results = pd.DataFrame(columns=['training_score', 'test_score'])
fprs, tprs, scores = [], [], []
X=train_x
y=train_y
clf=model
for (train, test), i in zip(cv.split(X,y), range(5)):
    model.fit(X.iloc[train], y.iloc[train])
    _, _, auc_score_train = compute_roc_auc(train)
    fpr, tpr, auc_score = compute_roc_auc(test)
    scores.append((auc_score_train, auc_score))
    fprs.append(fpr)
    tprs.append(tpr)

plot_roc_curve(fprs, tprs);
pd.DataFrame(scores, columns=['AUC Train', 'AUC Test'])

